In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd drive/MyDrive/projects/OCOR

/content/drive/MyDrive/projects/OCOR


验证当前环境：

In [4]:
!pwd
# !pip list

import torch, mmcv, numpy
print(torch.__version__)         # eg. 2.0.1
print(torch.version.cuda)        # eg. 11.8
print(mmcv.__version__)
print(numpy.__version__)

/home/featurize/work/OCOR
2.2.2+cu121
12.1
1.7.2
1.26.4


/environment/miniconda3/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
# ========================== 1. 使用 GPU ==========================
# 如果你不是 GPU 环境，点击上方菜单：修改 > 笔记本设置 > 硬件加速器 选 GPU
!nvidia-smi


Wed Jul 16 08:19:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080        Off |   00000000:10:00.0 Off |                  N/A |
| 30%   22C    P8             11W /  320W |       2MiB /  10240MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

# ========================== 2. 克隆 OCOR 项目 ==========================
# !git clone https://github.com/AliasJeff/OCOR.git
# %cd OCOR
# %ls


Cloning into 'OCOR'...
remote: Enumerating objects: 1093, done.
remote: Counting objects: 100% (972/972), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 1093 (delta 382), reused 941 (delta 369), pack-reused 121 (from 1)
Receiving objects: 100% (1093/1093), 7.40 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (389/389), done.
/content/OCOR
configs/        docs/            mmcv_custom/   requirements.txt  tests/
DatasetTest.py  evaluate_SOR.py  mmdet/         resources/        tools/
demo/           inference.py     README.md      setup.cfg         utils.py
docker/         LICENSE          requirements/  setup.py


In [23]:
!pip install scipy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 66.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 95.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.1 which is incompatible.
tensorflow 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.3.1 which is incompatible.


In [24]:
!pip uninstall -y numpy

# 安装 NumPy 1.x（避免 ABI 问题）
!pip install numpy==1.25.2


Found existing installation: numpy 2.3.1
Uninstalling numpy-2.3.1:
  Successfully uninstalled numpy-2.3.1
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 92.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.25.2 which is incompatible.


In [3]:

# ========================== 3. 安装依赖 ==========================

# Step 1: 清理旧依赖
!pip uninstall -y mmcv mmcv-full mmdet torch torchvision

# Step 2: 安装 PyTorch 2.0.1 + CUDA 11.8（与 mmcv-full 兼容）
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118

# Step 3: 安装 mmcv-full 1.7.1（适配 mmdet 2.12.0 和 torch 2.0）
# !pip install mmcv==1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
!pip install mmcv-full==1.7.2 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

# Step 4: 安装 MMDetection 2.12.0
!pip install mmdet==2.12.0

# 安装 ninja
!pip install ninja

Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.4 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 70.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 39.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89989 sha256=ece9551e48963d9c5ddcd45b98d7cd50b5ff7ff5123e941b2236e59cfa32a983
  Stored in directory: /home/featurize/.cache/pip/wheels/fc/5d/45/34fe

In [5]:
# 安装 mmdet + 本地项目（OCOR）
!pip install -e .

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///home/featurize/work/OCOR
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 103.6 MB/s eta 0:00:00
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.12.0
    Uninstalling mmdet-2.12.0:
      Successfully uninstalled mmdet-2.12.0
  Running setup.py develop for mmdet


In [ ]:


# ========================== 4. 下载预训练 Swin 模型 ==========================
!mkdir -p checkpoints
!wget https://download.openmmlab.com/mmclassification/v0/swin-transformer/swin-large_3rdparty_in21k_20220301-040dd71c.pth -O checkpoints/swin-large.pth


--2025-07-15 15:47:25--  https://download.openmmlab.com/mmclassification/v0/swin-transformer/swin-large_3rdparty_in21k_20220301-040dd71c.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.23.180, 47.246.23.183, 47.246.23.177, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.23.180|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-15 15:47:26 ERROR 404: Not Found.



In [6]:
!python setup.py develop

/environment/miniconda3/lib/python3.11/site-packages/setuptools/__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/environment/miniconda3/lib/python3.11/site-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 

In [27]:

# ========================== 5. 启动训练 ==========================
# 如果你有自定义数据目录，记得修改 config 文件中的数据路径
!python tools/train.py configs/ocor/ocor_swin_large_patch4_window7_fpn_300_proposals.py


/environment/miniconda3/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
apex is not installed
/environment/miniconda3/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
apex is not installed
apex is not installed
apex is not installed
>>> RUNTIME PYTHON: /environment/miniconda3/bin/python
>>> TORCH: 2.0.1+cu118
>>> NUMPY: 1.25.2
2025-07-16 08:50:07,068 - mmdet - INFO - Environment info:
---------------------------------

In [ ]:

# ========================== 6. 将训练结果保存到 Google Drive（可选） ==========================
from google.colab import drive
drive.mount('/content/drive')

# 将训练权重保存到 Google Drive
!cp work_dirs/ocor_swin_large_patch4_window7_fpn_300_proposals/latest.pth /content/drive/MyDrive/